In [6]:
import numpy as np
import csv
import os
import vtk

In [27]:
folder = 'C:/D/dev/Xnorm/reg_api/filters/include'
mind_pair_filename = os.path.join(folder, 'mind_pairs.csv')
mind_offset_filename = os.path.join(folder, 'mind_offsets.csv')

mind_pairs = []
with open(mind_pair_filename) as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for row in csvreader:
        mind_pairs.append((row[0], row[1]))
        

mind_offset = dict()
with open(mind_offset_filename) as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for row in csvreader:
        mind_offset[row[0]]=np.asarray(row[1:], dtype=np.float32)

print('MIND pairs \n', mind_pairs, '\n MIND offset\n', mind_offset)

MIND pairs 
 [('1', '2'), ('1', '5'), ('2', '5'), ('3', '6'), ('4', '6'), ('5', '6'), ('3', '4'), ('1', '4'), ('2', '3'), ('7', '5'), ('7', '6'), ('8', '9'), ('8', '10'), ('7', '8')] 
 MIND offset
 {'10': array([ 0., -1.,  0.], dtype=float32), '5': array([ 1.,  0.,  0.], dtype=float32), '2': array([ 0.5  , -0.866,  0.   ], dtype=float32), '7': array([ 0.,  0., -1.], dtype=float32), '9': array([ 0.,  1.,  0.], dtype=float32), '1': array([ 0.5  ,  0.866,  0.   ], dtype=float32), '3': array([-0.5  ,  0.866,  0.   ], dtype=float32), '4': array([-0.5  , -0.866,  0.   ], dtype=float32), '6': array([-1.,  0.,  0.], dtype=float32), '8': array([ 0.,  0.,  1.], dtype=float32)}


In [19]:
def create_a_line_polydata(p0, p1):
    
    points = vtk.vtkPoints()
    points.InsertNextPoint(p0)
    points.InsertNextPoint(p1)
    
    line = vtk.vtkLine()
    line.GetPointIds().SetId(0,0)
    line.GetPointIds().SetId(1,1)
    
    line_cell_array = vtk.vtkCellArray()
    line_cell_array.InsertNextCell(line)
    #for i in range(1):
    #    line = vtk.vtkLine()
    #    line.GetPointIds().SetId(0,i)
    #    line.GetPointIds().SetId(1, i+1)
    #    line_cell_array.InsertNextCell(line)    
        
    line_polydata = vtk.vtkPolyData()
    line_polydata.SetPoints(points)
    line_polydata.SetLines(line_cell_array)
    
    return line_polydata

def write_polydata_vtp(filename, polydata):
    
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(filename)
    if vtk.VTK_MAJOR_VERSION <= 5:
        writer.SetInput(polydata)
    else:
        writer.SetInputData(polydata)
    writer.Write()

In [23]:
def create_cube(center, bound):
    bounds =[]
    for i in range(3):
        bounds += [0, bound]
        
    cube = vtk.vtkCubeSource()
    cube.SetCenter(center)
    cube.SetBounds(bounds)
    cube.Update()
    
    return cube

In [21]:
folder = 'C:/Users/Ja/Google Drive/20180107_vtkline'

for indx, pair in enumerate(mind_pairs):
    if indx == 0 or indx == 6: continue
    p0, p1 = mind_offset[pair[0]], mind_offset[pair[1]]
    
    filename = os.path.join(folder, 'line_polydata%d.vtp' % indx)
    #filename = os.path.join(folder, 'line_polydata{}.vtp'.format(indx))
    line_polydata = create_a_line_polydata(p0, p1)
    write_polydata_vtp(filename, line_polydata)

    #print('Point p0, p1: ({}), ({})'.format(p0, p1)  )

In [29]:
for key, points in mind_offset.items():
    
    filename = os.path.join(folder, 'cube_polydata%s.vtp' % key)
    #print(points)
    cube = create_cube(points, bound=0.5)
    write_polydata_vtp(filename, cube.GetOutput())